In [122]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Latex, Math
import statsmodels.api as sm
%matplotlib inline

### Define the likelihood function

In [69]:
display(Math(r"L(\beta)=\sum_{i=1}^n P_i^{Y_i}(1-P_i)^{Y_i}"))

<IPython.core.display.Math object>

In [99]:
def likelihood(Y, pi):
    sum_total = 1
    sum_in = range(1,len(y)+1)
    for i in range(len(Y)):
        sum_in[i] = np.where(y[i]==1, pi[i],1-pi[i])
        sum_total *= sum_in[i]
    return sum_total

### Calculate probabilities for each observation

In [100]:
display(Math(r"P_i = P(x_i) = \frac{1}{1 + e^{-\sum_{j=0}^k\beta_j\cdot x_{ij}}}"))

<IPython.core.display.Math object>

In [101]:
def logitprobs(X,beta):
    n_rows = np.shape(X)[0]
    n_cols = np.shape(X)[1]
    pi=list(range(1,n_rows+1))
    expon=list(range(1,n_rows+1))
    for i in range(n_rows):
        expon[i] = 0
        for j in range(n_cols):
            ex=X[i][j] * beta[j]
            expon[i] = ex + expon[i]
        with np.errstate(divide="ignore", invalid="ignore"):
            pi[i]=1/(1+np.exp(-expon[i]))
    return pi   

### Calculated the diagonal matrix W

In [102]:
display(Math(r"W=diag(P_i\cdot (1-P_i))_{i=1}^n"))

<IPython.core.display.Math object>

In [103]:
def findW(pi):
    n = len(pi)
    W = np.zeros(n*n).reshape(n,n)
    for i in range(n):
        print(i)
        W[i,i] = (pi[i]*(1-pi[i])).astype(float)
    return W

### Obtain the solution for  the logistic function

In [104]:
display(Math(r"\beta_{n+1} = \beta_n -\frac{f(\beta_n)}{f'(\beta_n)}"))
display(Math(r" f(\beta)= X(Y-P)"))
display(Math(r" f(\beta)= XWX^T"))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [116]:
def logistics(X, Y, limit):
    import numpy as np
    from numpy import linalg
    nrow = np.shape(X)[0]
    bias = np.ones(nrow).reshape(nrow,1)
    X_new = np.append(X, bias, axis = 1)
    ncol = np.shape(X_new)[1]
    beta = np.zeros(ncol).reshape(ncol,1)
    root_dif = np.array(range(1,ncol+1)).reshape(ncol,1)
    iter_i = 10000
    while(iter_i>limit):
        print("Iter:i"+str(iter_i) + ", limit:" + str(limit))
        pi = logitprobs(X_new, beta)
        print("Pi:"+str(pi))
        W = findW(pi)
        print("W:"+str(W))
        num = (np.transpose(np.matrix(X_new))*np.matrix(Y - np.transpose(pi)).transpose())
        den = (np.matrix(np.transpose(X_new))*np.matrix(W)*np.matrix(X_new))
        root_dif = np.array(linalg.inv(den)*num)
        beta = beta + root_dif
        print("Beta: "+str(beta))
        iter_i = np.sum(root_dif*root_dif)
        ll = likelihood(Y, pi)
    return beta

## Prove the method

In [117]:
X = np.array(range(10)).reshape(10,1)
X

array([[0],
       [1],
       [2],
       [3],
       [4],
       [5],
       [6],
       [7],
       [8],
       [9]])

In [118]:
Y = [0,0,0,1,1,1,0,1,1,1]

In [119]:
bias = np.ones(10).reshape(10,1)
X_new = np.append(X,bias,axis=1)
X_new

array([[0., 1.],
       [1., 1.],
       [2., 1.],
       [3., 1.],
       [4., 1.],
       [5., 1.],
       [6., 1.],
       [7., 1.],
       [8., 1.],
       [9., 1.]])

### With stats 

In [123]:
logic_model = sm.Logit(Y,X_new)

In [124]:
result = logic_model.fit()

Optimization terminated successfully.
         Current function value: 0.431012
         Iterations 6


In [127]:
 (result.summary2())

<class 'statsmodels.iolib.summary2.Summary'>
"""
                        Results: Logit
==============================================================
Model:              Logit            No. Iterations:   6.0000 
Dependent Variable: y                Pseudo R-squared: 0.360  
Date:               2018-07-09 23:05 AIC:              12.6202
No. Observations:   10               BIC:              13.2254
Df Model:           1                Log-Likelihood:   -4.3101
Df Residuals:       8                LL-Null:          -6.7301
Converged:          1.0000           Scale:            1.0000 
----------------------------------------------------------------
         Coef.    Std.Err.      z      P>|z|     [0.025   0.975]
----------------------------------------------------------------
x1       0.6622     0.4001    1.6551   0.0979   -0.1220   1.4464
const   -2.2643     1.7038   -1.3290   0.1838   -5.6036   1.0750
==============================================================

"""